Importing the necessary libraries.  

In [4]:
import math
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score

Reading in the data file that was downloaded from Keeneland's website and condensing it down to just the 2024 Spring Meet.
Then, creating a new variable called 'Index' that will be used when merging with the 'main' data frame. Lastly, taking a look
at the top part of the data to make sure it was read in properly and getting the length of the data frame to make sure it is 
equal to the length of the 'main' data frame.

In [6]:
df = pd.read_csv("Keeneland Race Data Since Oct-2006.csv")
df = df[df["RaceMeet"] == "Spring 2024"]
df['Index'] = np.arange(len(df))
print(df.head())
print()
print("Length is " + str(len(df)))

         RaceMeet  RaceDate  RaceNumber               Surface RaceType  \
5295  Spring 2024  4/5/2024           1  Dirt                      STR   
5296  Spring 2024  4/5/2024           2  Dirt                      MSW   
5297  Spring 2024  4/5/2024           3  Turf                      ALW   
5298  Spring 2024  4/5/2024           4  Dirt                      MSW   
5299  Spring 2024  4/5/2024           5  Turf                      MSW   

      Distance  FieldSize  WinningPostPosition           HorseName  odds  ...  \
5295       7.0          7                    1  Play Good Pay Good   1.9  ...   
5296       4.5          9                    9       Shoot It True   0.5  ...   
5297       5.5          9                    3            Mo Stash   2.5  ...   
5298       7.0          9                    1         Roman Grace   2.8  ...   
5299       9.5         11                    4                Sy B   2.3  ...   

     LastWorkoutLocation  RouteSprint      WinnersSire  \
5295      

Reading in the 'main' data file that was made with data manually entered from a combination of Keeneland's daily race entries and results.
Checking to make sure there was no missing data before creating a subset of the 'main' data frame that only contains the horses that finished
1st. The middle 2 lines were provided by ChatGPT to help resolve a warning I was getting while trying to create an 'Index' variable like in 
the prior cell. Then, taking a look at top parts the 'main' and 'winners' data frames to make sure they look correct. Finally, getting the 
length of the 'winners' data frame to make sure it is equal to the length of previous data frame since that only contained data from the winning
horses.

In [8]:
df_main = pd.read_csv("Keeneland April 2024 Data for Capstone Project.csv")
print(df_main.isnull().values.any())
print()
winners = df_main[df_main["Finish"] == "1"]
winners = winners.copy()
winners.loc[:, 'Index'] = np.arange(len(winners))
print(df_main.head())
print(winners.head())
print()
print("Length is " + str(len(winners)))

False

   Day Weekday  Race Number  Post Position  M/L Odds Finish Final Odds  \
0    1  Friday            1              1       3.5      1       1.88   
1    1  Friday            1              2      15.0      6      12.46   
2    1  Friday            1              3       4.5      7       6.91   
3    1  Friday            1              4       8.0      S          S   
4    1  Friday            1              5      12.0      3       6.56   

   Change In Odds  
0           -1.62  
1           -2.54  
2            2.41  
3            0.00  
4           -5.44  
    Day Weekday  Race Number  Post Position  M/L Odds Finish Final Odds  \
0     1  Friday            1              1       3.5      1       1.88   
20    1  Friday            2             12       4.0      1       0.46   
24    1  Friday            3              3       2.0      1        2.5   
32    1  Friday            4              1       3.0      1        2.8   
46    1  Friday            5              5       8.0

Since both created data frames have the same length of 150, they can now be merged using the 'Index' variables that were created. 
Then, condensing the merged data frame down to the columns listed below and checking the data type of each column. The Final Odds 
column needed to be converted from a string data type to a float data type, but the other columns had the appropiate data type based 
on values in each column. The calculated percentage of winners that were favorites is 38.67%.

In [10]:
df_winners = pd.merge(left=winners,right=df,on='Index')
df_winners = df_winners[['Day','Weekday','Race Number','Post Position','M/L Odds','Final Odds','Change In Odds',
                       'Distance','FieldSize','Favorite','HalfMileTime','FinalTime','WinnersTrainer',
                       'WinnersJockey','ExactaPayout','SuperfectaPayout','TrifectaPayout']]
print(df_winners.head())
print()
df_winners['Final Odds'] = df_winners['Final Odds'].astype('Float64')
print(df_winners.convert_dtypes().dtypes)
print()
winners_as_favs = df_winners[df_winners["Favorite"] == "Y"]
print(len(winners_as_favs)/len(winners))

   Day Weekday  Race Number  Post Position  M/L Odds Final Odds  \
0    1  Friday            1              1       3.5       1.88   
1    1  Friday            2             12       4.0       0.46   
2    1  Friday            3              3       2.0        2.5   
3    1  Friday            4              1       3.0        2.8   
4    1  Friday            5              5       8.0       2.31   

   Change In Odds  Distance  FieldSize Favorite  HalfMileTime  FinalTime  \
0           -1.62       7.0          7        Y         45.99      88.86   
1           -3.54       4.5          9        Y         45.16      52.04   
2            0.50       5.5          9        Y         45.16      63.27   
3           -0.20       7.0          9        N         45.63      84.56   
4           -5.69       9.5         11        Y         49.02     118.86   

                                      WinnersTrainer  \
0  Dutrow Jr. Richard                            ...   
1  Ward Wesley              

Writing the merged data frame to a CSV file for use in Tableau.

In [12]:
df_winners.to_csv("merged_data_sets.csv")

Calculating the number of times a certain Post Position or Jockey or Trainer won during the meet.

In [14]:
winning_post_positions = df_winners['Post Position'].value_counts()
print(winning_post_positions)
print()
winning_jockey = df_winners['WinnersJockey'].value_counts()
print(winning_jockey)
print()
winning_trainer = df_winners['WinnersTrainer'].value_counts()
print(winning_trainer)

Post Position
1     19
5     18
6     16
2     16
9     14
4     12
10    12
8     11
7     11
3      9
12     6
11     4
14     2
Name: count, dtype: int64

WinnersJockey
Ortiz Jr. Irad                                                                                                             20
Ortiz Jose                                                                                                                 15
Gaffalione Tyler                                                                                                           15
Prat Flavien                                                                                                               10
Saez Luis                                                                                                                  10
Rosario Joel                                                                                                                9
Velazquez John                                                          

Calculating average FieldSize by Weekday (Sunday, Wednesday, Thursday, Friday, and Saturday only), average HalfMileTime and FinalTime
by distance in furlongs, and the average Morning Line odds/final odds/change in odds for a winner.

In [16]:
average_field_size_by_weekday = df_winners.groupby("Weekday")["FieldSize"].mean()
print(average_field_size_by_weekday)
print()
average_halfmile_by_distance = df_winners.groupby("Distance")["HalfMileTime"].mean()
print(average_halfmile_by_distance)
print()
average_finaltime_by_distance = df_winners.groupby("Distance")["FinalTime"].mean()
print(average_finaltime_by_distance)
print()
average_winner_mlodds = df_winners["M/L Odds"].mean()
print("The average Morning Line odds of a winner: " + str(average_winner_mlodds))
print()
average_winner_finalodds = df_winners["Final Odds"].mean()
print("The average final odds of a winner: " + str(average_winner_finalodds))
print()
average_winner_oddschange = average_winner_finalodds - average_winner_mlodds
print("The average change in odds of a winner: " + str(average_winner_oddschange))

Weekday
Friday       8.900000
Saturday     9.187500
Sunday       9.407407
Thursday     7.814815
Wednesday    7.958333
Name: FieldSize, dtype: float64

Distance
4.5     46.052222
5.5     44.540625
6.0     45.798261
6.5     45.226000
7.0     45.953200
8.0     47.502500
8.5     47.993514
9.0     47.939000
9.5     48.112857
12.0    49.865000
Name: HalfMileTime, dtype: float64

Distance
4.5      52.746667
5.5      62.973125
6.0      71.418261
6.5      77.478000
7.0      85.743200
8.0      95.986667
8.5     105.181892
9.0     111.377000
9.5     116.787143
12.0    148.875000
Name: FinalTime, dtype: float64

The average Morning Line odds of a winner: 6.5840000000000005

The average final odds of a winner: 5.688

The average change in odds of a winner: -0.8960000000000008


Calculating the number of scratches and creating a subset of the 'main' data frame without scratches. The Final Odds 
column and Finish column needed to be converted from a string data type to a float data type and integer data type,
respectively. Finally, taking a look at the top part of the newly created data frame to make sure it looks correct.

In [18]:
number_of_scratches = len(df_main[df_main["Finish"]=='S'])
print("The number of scratches: " + str(number_of_scratches))
print()
df_main_no_scratches = df_main[df_main["Finish"] != "S"]
df_main_no_scratches = df_main_no_scratches.copy()
df_main_no_scratches.loc[:, 'Final Odds'] = df_main_no_scratches['Final Odds'].astype('Float64')
df_main_no_scratches.loc[:, 'Finish'] = df_main_no_scratches['Finish'].astype('Int64')
df_main_no_scratches.loc[:, 'Change In Odds'] = np.abs(df_main_no_scratches['Change In Odds'])
print(df_main_no_scratches.convert_dtypes().dtypes)
print()
print(df_main_no_scratches.head())

The number of scratches: 333

Day                        Int64
Weekday           string[python]
Race Number                Int64
Post Position              Int64
M/L Odds                 Float64
Finish                     Int64
Final Odds               Float64
Change In Odds           Float64
dtype: object

   Day Weekday  Race Number  Post Position  M/L Odds Finish Final Odds  \
0    1  Friday            1              1       3.5      1       1.88   
1    1  Friday            1              2      15.0      6      12.46   
2    1  Friday            1              3       4.5      7       6.91   
4    1  Friday            1              5      12.0      3       6.56   
6    1  Friday            1              7       6.0      4       9.39   

   Change In Odds  
0            1.62  
1            2.54  
2            2.41  
4            5.44  
6            3.39  


Writing the above data frame to a CSV file for use in Tableau.

In [20]:
df_main_no_scratches.to_csv('main_no_scratches.csv')

Calculating the averages of Morning Line Odds, Final Odds, and Change In Odds by Finish. The Final Odds is closest
to being a 'perfect' 1 to 12 in order.

In [22]:
average_mlodds_by_finish = df_main_no_scratches.groupby("Finish")["M/L Odds"].mean().sort_values(ascending=True)
print(average_mlodds_by_finish)
print()
average_finalodds_by_finish = df_main_no_scratches.groupby("Finish")["Final Odds"].mean().sort_values(ascending=True)
print(average_finalodds_by_finish)
print()
average_change_in_odds_by_finish = df_main_no_scratches.groupby("Finish")["Change In Odds"].mean().sort_values(ascending=True)
print(average_change_in_odds_by_finish)

Finish
1      6.584000
2      7.770667
3      9.344000
4     11.806000
5     12.037162
6     13.487143
9     14.709877
7     14.900877
12    15.125000
8     15.352041
10    19.338983
11    20.666667
Name: M/L Odds, dtype: float64

Finish
1         5.688
2      8.063733
3     10.069733
4     14.496667
5     17.800068
6     20.007286
7     23.873509
8     27.414082
9      28.06642
12    33.834167
10    42.658814
11    47.208571
Name: Final Odds, dtype: object

Finish
1      3.570667
2      3.732533
3      4.673600
4      7.127467
6      9.056857
5      9.057905
7     11.443860
8     14.402245
9     14.634321
12    19.347500
10    23.904576
11    26.871905
Name: Change In Odds, dtype: float64


Creating a function to be used for splitting odds or changes in odds into 8 groups. I based the cut-off values on typically
seen Morning Line odds. I wanted to see if big odds or odds changes had a lopsided effect when creating models to compare. 

In [24]:
def splitter(s):
    li = []
    for i in s:
        if i <= 2:
            li.append(1) 
        elif i <= 5:
            li.append(2)
        elif i <= 10:
            li.append(3)
        elif i <= 15:
            li.append(4)
        elif i <= 20:
            li.append(5)
        elif i <= 30:
            li.append(6)
        elif i <= 50:
            li.append(7)
        else:
            li.append(8)
    return li

Using lambda functions to create 3 new variables, which are the Morning Line Odds, Final Odds, and Change In Odds after 
being split into the appropriate group. NOTE: the values in the Diff_Odds_Split column do not need to be equal to the
difference in the corresponding ML_Odds_Split and F(inal)_Odds_Split values.

In [26]:
df_main_no_scratches = df_main_no_scratches.assign(ML_Odds_Split = lambda x: splitter(df_main_no_scratches["M/L Odds"]))
df_main_no_scratches = df_main_no_scratches.assign(F_Odds_Split = lambda x: splitter(df_main_no_scratches["Final Odds"]))
df_main_no_scratches = df_main_no_scratches.assign(Diff_Odds_Split = lambda x: splitter(df_main_no_scratches["Change In Odds"]))
print(df_main_no_scratches.head())

   Day Weekday  Race Number  Post Position  M/L Odds Finish Final Odds  \
0    1  Friday            1              1       3.5      1       1.88   
1    1  Friday            1              2      15.0      6      12.46   
2    1  Friday            1              3       4.5      7       6.91   
4    1  Friday            1              5      12.0      3       6.56   
6    1  Friday            1              7       6.0      4       9.39   

   Change In Odds  ML_Odds_Split  F_Odds_Split  Diff_Odds_Split  
0            1.62              2             1                1  
1            2.54              4             4                2  
2            2.41              2             3                2  
4            5.44              4             3                3  
6            3.39              3             3                2  


Calculating the averages of Morning Line Odds (after being split), Final Odds (after being split), and 
Change In Odds (after being split) by Finish. The Final Odds (after being split) is again closest to being 
a 'perfect' 1 to 12 in order.

In [28]:
average_smlodds_by_finish = df_main_no_scratches.groupby("Finish")["ML_Odds_Split"].mean().sort_values(ascending=True)
print(average_smlodds_by_finish)
print()
average_sfinalodds_by_finish = df_main_no_scratches.groupby("Finish")["F_Odds_Split"].mean().sort_values(ascending=True)
print(average_sfinalodds_by_finish)
print()
average_schange_in_odds_by_finish = df_main_no_scratches.groupby("Finish")["Diff_Odds_Split"].mean().sort_values(ascending=True)
print(average_schange_in_odds_by_finish)

Finish
1     2.493333
2     2.753333
3     3.006667
4     3.466667
5     3.540541
6     3.707143
9     3.950617
7     3.964912
8     4.051020
12    4.083333
10    4.610169
11    4.690476
Name: ML_Odds_Split, dtype: float64

Finish
1     2.306667
2     2.733333
3     3.200000
4     3.846667
5     4.209459
6     4.371429
7     4.894737
8     5.204082
9     5.283951
12    5.791667
10    6.220339
11    6.428571
Name: F_Odds_Split, dtype: float64

Finish
2     1.766667
1     1.813333
3     2.100000
4     2.553333
5     2.783784
6     2.885714
7     3.166667
8     3.459184
9     3.740741
12    4.208333
10    4.813559
11    4.928571
Name: Diff_Odds_Split, dtype: float64


Writing the main_no_scratches data frame, after being updated to include the 'split' columns, to a CSV file 
for use in Tableau.

In [30]:
df_main_no_scratches.to_csv('main_no_scratches_grouped.csv')

The following sections are a practice attempt at using scikit-learn for Linear Regression to determine if 
Morning Line odds or Final odds or the change in odds are better for determing a horse's finishing position. 
These 2 websites helped with the code and provide a better explaination, than I am currently capable of, of what is being done:
https://medium.com/@neslihannavsar/building-a-linear-regression-model-with-scikit-learn-part-1-eed4c04f53f9
https://www.geeksforgeeks.org/python-linear-regression-using-sklearn/#

You can switch the # from line 3 to line 2 to run the 'split' version. Make sure to do the same for the other models
so all 3 models are the same version when comparing them.

In [33]:
#M/L Odds model
X = np.array(df_main_no_scratches['M/L Odds']).reshape(-1, 1)
#X = np.array(df_main_no_scratches['ML_Odds_Split']).reshape(-1, 1)
y = np.array(df_main_no_scratches['Finish']).reshape(-1, 1)
ml_model = LinearRegression().fit(X,y)
print ('Intercept: ',ml_model.intercept_)    
print ('Coefficients: ', ml_model.coef_[0])
# MSE
y_pred = ml_model.predict(X)
mse = mean_squared_error(y, y_pred)
print(mse)
y.mean() 
y.std() 
# RMSE
print(np.sqrt(mse))
# MAE
print(mean_absolute_error(y, y_pred))
# R2 Score
print(ml_model.score(X, y))

Intercept:  [3.65959736]
Coefficients:  [0.12381382]
7.446915895319539
2.7289037900445554
2.2604210065444357
0.1473245078412827


You can switch the # from line 3 to line 2 to run the 'split' version.

In [35]:
#Final Odds model
X = np.array(df_main_no_scratches['Final Odds']).reshape(-1, 1)
#X = np.array(df_main_no_scratches['F_Odds_Split']).reshape(-1, 1)
f_model = LinearRegression().fit(X,y)
print ('Intercept: ',f_model.intercept_)    
print ('Coefficients: ', f_model.coef_[0])
# MSE
y_pred = f_model.predict(X)
mse = mean_squared_error(y, y_pred)
print(mse)
y.mean() 
y.std() 
# RMSE
print(np.sqrt(mse))
# MAE
print(mean_absolute_error(y, y_pred))
# R2 Score
print(f_model.score(X, y))

Intercept:  [3.98406933]
Coefficients:  [0.06319424]
6.817208486263278
2.6109784538106164
2.148719577614339
0.21942631246492683


You can switch the # from line 3 to line 2 to run the 'split' version.

In [37]:
#Change In Odds model
X = np.array(df_main_no_scratches['Change In Odds']).reshape(-1, 1)
#X = np.array(df_main_no_scratches['Diff_Odds_Split']).reshape(-1, 1)
cio_model = LinearRegression().fit(X,y)
print ('Intercept: ',cio_model.intercept_)    
print ('Coefficients: ', cio_model.coef_[0])
# MSE
y_pred = cio_model.predict(X)
mse = mean_squared_error(y, y_pred)
print(mse)
y.mean() 
y.std() 
# RMSE
print(np.sqrt(mse))
# MAE
print(mean_absolute_error(y, y_pred))
# R2 Score
print(cio_model.score(X, y))

Intercept:  [4.41358927]
Coefficients:  [0.07810487]
7.434123532322491
2.726558917816098
2.2655812448146824
0.14878923962658952


Double click this markdown cell to make it display correctly.

Summary of Models:
                            Pre-split    Post-split
Morning Line Odds     RMSE    2.73          2.70
                      MAE     2.26          2.23
                      R^2     0.15          0.17

Final Odds            RMSE    2.61          2.51
                      MAE     2.15          2.05
                      R^2     0.22          0.28

Change In Odds        RMSE    2.73          2.67
                      MAE     2.27          2.21
                      R^2     0.15          0.18

RMSE = Root Mean Squared Error, lower is better
MAE  = Mean Absolute Error, lower is better
R^2  = R-squared, closer to 1 is better

The Final Odds Post-split model is the best of these 6 models. The low R-squared values of the models show that
there are more variables that help determine the finishing position of a horse than just its odds or change in odds.